#### port_lite database: sales, stocks table
#### csv files: daily-sales, monitors, stocks-all

In [1]:
import calendar
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

today = date.today()
yesterday = today - timedelta(days=1)
yesterday

datetime.date(2022, 1, 4)

### Yesterday = Last closed business day

In [4]:
#yesterday = today - timedelta(days=1)
yesterday = date(2022, 1, 5)
yesterday

datetime.date(2022, 1, 5)

### Restart and Run All Cells

In [5]:
format_dict = {'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
               'created_at':'{:%Y-%m-%d %H:%M:%S}',
               'fm_price':'{:,.2f}','to_price':'{:,.2f}','diff':'{:,.2f}',
               'max_price':'{:,.2f}','min_price':'{:,.2f}',
               'buy_target':'{:,.2f}','sell_target':'{:,.2f}',
               'volume':'{:,.2f}','beta':'{:,.2f}','cost':'{:,.2f}',
               'qty':'{:,}','available_qty':'{:,}',
               'pct':'{:,.2f}%','percent':'{:,.2f}%'
              }

In [6]:
sql = '''
SELECT * FROM sales ORDER BY id DESC LIMIT 1
'''
tmp = pd.read_sql(sql, conlite)
tmp

,id,name,fm_date,to_date,days,fm_price,to_price,diff,pct,ttl_spread,avg_spread,max_price,min_price,qty,created_at,updated_at,latest_date_id
0,1082700,BGC,2022-01-04,2022-01-05,1,10.7,10.6,-0.1,-0.93,-1,-1,10.7,10.6,1222340,2022-01-05 12:21:05.706563,2022-01-05 12:21:05.706563,1


In [7]:
names = tmp['name']
name = names.to_string(index=False)
sql = '''
SELECT * FROM stocks
WHERE name = '%s'
'''
sql = sql % name
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.style.format(format_dict)


SELECT * FROM stocks
WHERE name = 'BGC'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,689,BGC,13.10,9.20,O,10.40,0.00,5.79,0.69,0.00,"9,000",-4,4,0,0,0,DOS,SET


In [8]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
round((to_price-fm_price)/fm_price*100,2) AS percent,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a JOIN stocks t ON a.name = t.name 
WHERE to_date >= "%s" AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)

df = pd.read_sql(sql, conlite)
df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.head().style.format(format_dict)


SELECT a.name,fm_date,to_date,fm_price,to_price,
round((to_price-fm_price)/fm_price*100,2) AS percent,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a JOIN stocks t ON a.name = t.name 
WHERE to_date >= "2022-01-05" AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



,fm_date,to_date,fm_price,to_price,percent,qty,max_price,min_price,status,market
name,,,,,,,,,,
GLOBAL,2021-12-30,2022-01-05,20.00,20.10,0.50%,"15,157,998",20.20,20.00,B,SET50
IMH,2022-01-04,2022-01-05,12.20,13.70,12.30%,"6,346,286",13.80,12.30,B,mai
KCE,2021-12-30,2022-01-05,88.00,88.50,0.57%,"22,028,988",89.25,85.75,B,SET50
NOBLE,2021-12-28,2022-01-05,6.10,6.25,2.46%,"12,239,436",6.25,6.10,B,SET
PTT,2022-01-04,2022-01-05,38.00,39.00,0.00%,"76,595,895",39.00,38.00,B,SET50


### Price change up or down more than 3%

In [9]:
#df['bdays'] = np.busday_count(df['fm_date'].dt.date,df['to_date'].dt.date)
#df['bdays'] = 1
df['bdays'] = (df['to_date'] - df['fm_date']).dt.days
watch_list = df.loc[
    abs(df.percent) >= 3,
    ['percent','status','fm_price','to_price','market','fm_date','bdays','max_price','min_price']
]
watch_list.sort_values(by="percent",ascending=False).style.format(format_dict)

,percent,status,fm_price,to_price,market,fm_date,bdays,max_price,min_price
name,,,,,,,,,
IMH,12.30%,B,12.20,13.70,mai,2022-01-04,1,13.80,12.30
SAT,9.43%,I,21.20,23.20,SET,2021-12-27,9,23.60,21.10
IVL,7.02%,I,42.75,45.75,SET50,2021-12-28,8,45.75,42.50
TCAP,6.08%,O,37.00,39.25,SET100,2021-12-27,9,39.25,37.00
TOP,5.05%,I,49.50,52.00,SET50,2021-12-30,6,52.75,49.50
DCC,4.96%,I,2.82,2.96,SET,2021-12-27,9,3.00,2.82
TISCO,4.77%,B,94.25,98.75,SET50,2021-12-24,12,99.25,94.50
ROJNA,3.85%,O,6.50,6.75,SET,2021-12-24,12,6.75,6.50
SIS,-4.71%,B,47.75,45.50,SET,2022-01-04,1,48.25,45.00


In [10]:
file_name = "daily-watch-list.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

watch_list.sort_values(['percent'],ascending=[True]).to_csv(output_file)
watch_list.sort_values(['percent'],ascending=[True]).to_csv(data_file)
watch_list.sort_values(['percent'],ascending=[True]).to_csv(box_file)

### Create daily-sales from sales

In [11]:
df.head().style.format(format_dict)

,fm_date,to_date,fm_price,to_price,percent,qty,max_price,min_price,status,market,bdays
name,,,,,,,,,,,
GLOBAL,2021-12-30,2022-01-05,20.00,20.10,0.50%,"15,157,998",20.20,20.00,B,SET50,6
IMH,2022-01-04,2022-01-05,12.20,13.70,12.30%,"6,346,286",13.80,12.30,B,mai,1
KCE,2021-12-30,2022-01-05,88.00,88.50,0.57%,"22,028,988",89.25,85.75,B,SET50,6
NOBLE,2021-12-28,2022-01-05,6.10,6.25,2.46%,"12,239,436",6.25,6.10,B,SET,8
PTT,2022-01-04,2022-01-05,38.00,39.00,0.00%,"76,595,895",39.00,38.00,B,SET50,1


In [12]:
file_name = "daily-sales.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

df.sort_values(['status','name'],ascending=[True,True]).to_csv(output_file,header=None)
df.sort_values(['status','name'],ascending=[True,True]).to_csv(data_file,header=None)
df.sort_values(['status','name'],ascending=[True,True]).to_csv(box_file)

### Create monitors from stocks

In [13]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("B","I","O","S")
ORDER BY name
"""
monitors = pd.read_sql(sql, conlite)
monitors.set_index(["name"], inplace=True)
monitors.head()

,status
name,
BGC,O
BGRIM,I
DCC,I
DOHOME,I
EPG,I


In [14]:
sql = """
SELECT status, COUNT(*) AS 'no. of stocks'
FROM stocks 
WHERE status IN ("B","I","O","S")
GROUP BY status
"""
df_tmp = pd.read_sql(sql, conlite)
df_tmp

,status,no. of stocks
0,B,10
1,I,10
2,O,4


### output_file = iCloud Drive on PC

In [15]:
file_name = "monitors.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

monitors.to_csv(data_file, header=None)
monitors.to_csv(box_file)
monitors.to_csv(output_file)

### End of daily process

In [16]:
sql = """
SELECT status, COUNT(*) AS 'no. of stocks'
FROM stocks 
GROUP BY status
"""
df_tmp = pd.read_sql(sql, conlite)
df_tmp

,status,no. of stocks
0,B,10
1,I,10
2,O,4
3,T,5
4,U,9
5,X,14


In [17]:
sql = """
SELECT name, status, market
FROM stocks 
ORDER BY status, name
"""
stocks = pd.read_sql(sql, conlite)
stocks.set_index(["name"], inplace=True)
stocks

,status,market
name,,
GLOBAL,B,SET50
IMH,B,mai
KCE,B,SET50
NOBLE,B,SET
PTT,B,SET50
RATCH,B,SET50
SIS,B,SET
STA,B,SET50
TISCO,B,SET50


In [18]:
file_name = "stocks-all.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

stocks.to_csv(data_file, header=None)
stocks.to_csv(output_file)
stocks.to_csv(box_file)

In [19]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("B","O")
ORDER BY name
"""

buy_candidates = pd.read_sql(sql, conlite)
buy_candidates.set_index(["name"], inplace=True)
buy_candidates

,status
name,
BGC,O
GLOBAL,B
IMH,B
KCE,B
NOBLE,B
PTT,B
RATCH,B
ROJNA,O
SIS,B


In [20]:
buy_candidates.shape[0]

14

In [21]:
sql = """
SELECT name, status 
FROM stocks 
WHERE status IN ("I","S")
ORDER BY name
"""

sell_candidates = pd.read_sql(sql, conlite)
sell_candidates.set_index(["name"], inplace=True)
sell_candidates

,status
name,
BGRIM,I
DCC,I
DOHOME,I
EPG,I
IVL,I
KBANK,I
RJH,I
SAT,I
TOP,I


In [22]:
sell_candidates.shape[0]

10